# Laboratorio 4

## Equipo U

#### Ana Laura Correa Cabrales
#### Carlos Eduardo Ochoa Arambula
#### Pablo Francisco Vargas Castellanos
#### Emilio De La Torre Jimenez

El propósito: Que puedas describir los efectos cuantitativos en el precio durante los comunicados de indicadores macroeconómicos, así como el detectar oportunidades de inversión a partir de los resultados obtenidos. Lo anterior, utilizando lo que se conoce como "Calendario Económico", y más en particular, aquel que incorpora información de las principales economías del mundo por lo que es ampliamiente utilizado en el mercado de Forex.

In [164]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as py
from collections import Counter
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

In [165]:
#Cargamos los datos, y eliminamos los que no tienen valor
df = pd.read_csv("calendario_economico.csv") 
df.dropna(inplace = True)

In [166]:
#buscamos y agrupamos nuestros datos en Data Frames
data1=df.loc[df['Name'] == 'Industrial Production s.a. (MoM)']
IndProduction=data1.loc[df['Country'] == 'European Monetary Union']
IndProduction=IndProduction.reset_index(drop=True)
data2=df.loc[df['Name'] == 'Retail Sales (MoM)']
RetailSales=data2.loc[df['Country'] == 'European Monetary Union']
RetailSales=RetailSales.reset_index(drop=True)


In [167]:
#Date time a la variable "timestamp" para poder despues buscarla en los precios 
IndProduction['timestamp']=pd.to_datetime(IndProduction['timestamp'])
IndProduction['timestamp']=IndProduction['timestamp'].dt.tz_localize('UTC')
RetailSales['timestamp']=pd.to_datetime(RetailSales['timestamp'])
RetailSales['timestamp']=RetailSales['timestamp'].dt.tz_localize('UTC')

In [168]:
#definimos escenarios por restricciones
def escenario(x):
    x['escenario']= x['actual']
    for i in range(len(x)):
        if x['actual'][i] >= x['consensus'][i] >= x['previous'][i]:
            x['escenario'][i] = "A"
        elif x['actual'][i] >= x['consensus'][i] < x['previous'][i]:
            x['escenario'][i]="B"
        elif x['actual'][i] < x['consensus'][i] >= x['previous'][i]:
            x['escenario'][i]="C"
        elif x['actual'][i] < x['consensus'][i] < x['previous'][i]:
            x['escenario'][i]="D"
    return x
    

In [169]:
RS=escenario(RetailSales)
IP=escenario(IndProduction)

In [170]:
#cargamos los datos de los precios del Euro-Dolar
df = pd.read_csv("precios_historicos_eurusd.csv") 
df['timestamp']=pd.to_datetime(df['timestamp'])
df['timestamp']=df['timestamp'].dt.tz_localize('UTC')

In [214]:
#checando valores en precios EUR/USD con misma hora que nuestro calendario economico
#Data frame con los valores que coinciden las horas
indices=df[df.timestamp.isin(RS.timestamp)]
#tomar los indices de media hora antes y despues



timestamp    2009-01-09 10:00:00+00:00
open                           1.36941
high                           1.37326
low                            1.36927
close                          1.37226
Name: 924, dtype: object

AttributeError: module 'pandas' has no attribute 'indices'